# Chapter 12 Sorting and Selection
## 12.1 Why Study Sorting Algorithms?
이번 챕터에서는 객체들의 집합을 정렬(sort)하는 정렬 알고리즘을 배운다. 정렬은 가장 중요하면서도 잘 연구된 문제 중 하나이다. 데이터들은 이진 탐색 알고리즘을 이용하여 효율적인 탐색을 실시하기 위해 정렬되기도 하며, 다양한 문제들을 위한 다양한 고급 알고리즘들이 정렬을 이용하고 있다. 파이썬은 `list` 클래스를 정렬하기 위한 built-in `sort` 함수, 정렬된 리스트를 반환해주는 `sorted` 함수를 지원한다. 이 함수들은 고급 알고리즘을 이용하며(이 중 몇몇은 이번 챕터에서 다룰 것이다), 고도로 최적화되어있다. 프로그래머들은 일반적으로 이러한 built-in 정렬 함수를 이용하고, 직접 처음부터 정렬 알고리즘을 구현해야 하는 상황은 극히 드물다.

그렇다면 왜 정렬 알고리즘에 대한 깊은 이해가 필요할까? 당장 built-in 함수를 이용하더라도, 그 효율성이 어느정도인지, 원소들이 처음에 어떻게 정렬되어 있는지에 효율성이 어떤 영향을 받는지, 원소들의 타입은 얼마나 중요한지 알면 좋을 것이다. 또, 정렬 알고리즘의 발전에 기여한 아이디어와 접근방식들은 컴퓨팅 분야의 다른 알고리즘 발전에도 중요한 역할을 했다. 우리는 이미 다음의 정렬 알고리즘들을 다룬 적이 있다.
- Insertion-sort(Section 5.5.2, 7.5, 9.4.1)
- Selection-sort(Section 9.4.1)
- Bubble-sort(Exercise C-7.38)
- Heap-sort(Section 9.4.2)

이번 챕터에서는 또 다른 4개의 정렬 알고리즘, **merge-sort**, **quick-sort**, **bucket-sort**, **radix-sort**를 살펴보고, Section 12.5에서는 다양한 알고리즘들의 장점과 단점을 살펴볼 것이다.

### 12.2.1 Divide-and-conquer
merge-sort와 quick-sort는 **분할 정복(divide-and-conquer)**이라고 불리는 알고리즘 디자인 패턴의 재귀(recursion)를 이용한다. 
1. **Divide:** 만약 입력 크기가 일정 기준(threshold)보다 작다면 직접적인 메소드를 이용하여 문제를 풀고 그 결과로 얻은 답을 반환한다. 그렇지 않다면 입력 데이터를 서로 다른 두 부분집합으로 분할한다.
2. **Conquer** 부분집합의 문제를 재귀적으로 해결한다.
3. **Combine** 위의 과정에서 얻은 subproblem의 답(solution)을 모아서 원래의 문제에 대한 답으로 합친다(merge).

### Using Divide-and-Conquer for Sorting
우선 merge-sort 알고리즘을 high-level에서 살펴보자.
1. **Divide:** 만약 $S$가 0개나 1개의 원소를 갖는다면 이미 정렬되어 있는 것이므로 $S$를 반환한다. $S$가 2개 이상의 원소를 갖는다면 $S$의 원소를 반씩 나눠갖는 두 개의 시퀀스 $S_{1}$, $S_{2]$로 분할한다. 이 때 $S_{1}$은 $S$의 $\lfloor n/2 \rfloor$ 원소를 저장하고, $S_{2}$는 $S$의 $\lceil n/2 \rceil$ 원소를 저장한다.
2. **Conquer:** 재귀적으로 $S_{1}$과 $S_{2}$를 정렬한다.
3. **Combine:** 정렬된 시퀀스 $S_{1}$와 $S_{2}$를 $S$에 합치면서 정렬된 시퀀스로 만든다.

이진 트리 $T$를 이용해서 merge-sort 알고리즘의 실행 과정을 **merge-sore tree**로 표현할 수 있다. 아래의 그림들을 쭉 보면 된다.

<img width="347" alt="figure-12.1" src="https://user-images.githubusercontent.com/20944657/37184091-dc0ad8fc-237c-11e8-8432-933487dcc284.png">
<img width="640" alt="figure-12.2" src="https://user-images.githubusercontent.com/20944657/37184094-e113c890-237c-11e8-81bd-b775d0bbacbf.png">
<img width="635" alt="figure-12.3" src="https://user-images.githubusercontent.com/20944657/37184095-e224caea-237c-11e8-8ef6-f9fca66f19b0.png">
<img width="637" alt="figure-12.4" src="https://user-images.githubusercontent.com/20944657/37184096-e358879e-237c-11e8-8acf-f4d9057f2660.png">

**Proposition 12.1:** 크기가 $n$인 시퀀스에 merge-sort를 실행할 때 이를 표현한 merge-sort tree의 높이는 $\lceil \log n \rceil$이다.

위 명제의 증명은 쉬우므로 생략한다.

### 12.2.2 Array-Based Implementation of Merge-Sort
이제 파이썬의 (배열 기반) `list`를 이용해서 merge-sort를 구현해보자.

In [1]:
def merge(S1, S2, S):
    """Merge two sorted Python lists S1 and S2 into properly sized list S."""
    i = j = 0
    while i + j < len(S):
        if j == len(S2) or (i < len(S1) and S1[i] < S2[j]):
            S[i+j] = S1[i]      # copy ith element of S1 as next item of S
            i += 1
        else:
            S[i+j] = S2[j]      # copy jth element of S2 as next item of S
            j += 1
            
def merge_sort(S):
    """Sort the elements of Python list S using the merge-sort algorithm."""
    n = len(S)
    if n < 2:
        return                        # list is already sorted
    # divide
    mid = n // 2
    S1 = S[0:mid]                     # copy of first half
    S2 = S[mid:n]                     # copy of second half
    # conquer (with recursion)
    merge_sort(S1)                    # sort copy of first half
    merge_sort(S2)                    # sort copy of second half
    # merge results
    merge(S1, S2, S)                  # merge sorted halves back into S

<img width="700" alt="figure-12.5" src="https://user-images.githubusercontent.com/20944657/37190795-3abff22a-239f-11e8-9e03-e9f1eeb1fee1.png">


### 12.2.3 The Running Time of Merge-Sort
우선 `merge` 알고리즘의 작동 시간을 분석해보자. $S_{1}$과 $S_{2}$의 원소의 수를 $n_{1}$, $n_{2}$라 하면 `merge`의 시간 복잡도가 $O(n_{1} + n_{2})$인 것은 쉽게 알 수 있다. 그러면 이제 위에서 살펴봤던 merge-sort tree를 이용해서 merge-sort 알고리즘의 시간 복잡도를 분석해보자. 간단하게 분석하기 위해 $n$이 2의 지수 꼴임을 가정하지만 그렇지 않은 경우에도 똑같은 결과를 보일 수 있다(Exercise R-12.3).

재귀를 이용하는 알고리즘의 시간 복잡도를 분석할 때에는 각각의 재귀 호출에서 얼마만큼의 시간이 걸리는 지를 파악하고, 몇번의 호출이 일어나는지를 알아내서 전체 시간을 분석했었다. 이 방법을 그대로 이용하자. 각각의 $v$에 대한 merge_sort 호출에서는 분할과 merge가 이루어진다. 그런데 $v$의 분할에 걸리는 시간은 $v$의 길이에 비례할 것이 당연하고, `merge` 또한 그 길이에 비례한다는 것을 위에서 살펴봤다. 이제 노드 $v$의 깊이(depth)를 $i$라 할 때, $v$에서 걸리는 시간은 $O(n/2^{i})$가 되는데, 각각의 $v$의 길이는 $n/2^{i}$가 되기 때문이다.

이제 트리 $T$를 전체적으로 보자. 각각의 레벨 $i$에 있는 노드의 개수가 $2^{i}$개 이므로 각각의 레벨에서 소요되는 시간은 $2^{i} \cdot n / 2^{i} = n$이다. $T$의 높이가 $\lceil \log n \rceil + 1$이고 각각의 레벨에서 $O(n)$의 시간이 걸리므로 우리는 다음의 결과를 얻는다.

**Proposition 12.2:** 크기가 $n$인 시퀀스 $S$에 merge-sort 알고리즘을 적용했을 때의 시간복잡도는 $O(n\log n)$이다. 이 때 $S$의 두 원소는 $O(1)$ 시간 안에 비교될 수 있다고 가정한다.

<img width="640" alt="figure-12.6" src="https://user-images.githubusercontent.com/20944657/37185540-3317e0d4-2384-11e8-833f-e5c70df15eba.png">

### 12.2.4 Merge-Sort and Recurrence Equations
이번엔 **점화식(recurrence equation)**을 이용해서 merge-sort 알고리즘의 시간 복잡도가 $O(\log n)$임을 보여보자. 크기가 $n$인 입력 시퀀스에 merge-sort를 실행할 때 걸리는 worst-case 시간을 $t(n)$이라 하자. merge-sort가 recursive하므로 $t(n)$도 recursive하게 표현하는 것이 가능하다. 역시나 문제를 간단히 만들기 위해 $n$이 2의 지수 꼴임을 가정하자. 그러면, $t(n)$을 다음과 같이 표현하는 것이 가능하다.

$  
t(n) = 
\left\{
    \begin{array}{l}
      b && n \leq 1\\
      2t(n/2) + cn && otherwise.
    \end{array}
  \right.
$

이제 우리는 이를 이용해서 $t(n)$의 표현에 $t(n)$을 이용하지 않는 **closed-form** characterization을 구하고 $t(n)$을 big-Oh로 나타내야 한다. $n$이 큰 양수라고 가정하자. 그러면 점화식 $t(n)$을 다음과 같이 표현할 수 있다.

$t(n) = 2(2t(n/2^{2}) + (cn/2)) + cn\\
\hspace{0.7cm}= 2^{2}t(n/2^{2}) + 2(cn/2) + cn \\
\hspace{0.7cm}= 2^{2}t(n/2^{2}) + 2cn\\
\hspace{0.7cm}= 2^{3}t(n/2^{3}) + 3cn$

이제 규칙을 찾을 수 있을 것이다. 이렇게 식을 $i$번 적용하면 우리는 $t(n) = 2^{i}t(n/2^{i}) + icn$을 얻는다. 그러나 언제 이러한 과정을 멈춰야 할까? 위에서 $t(n) = b$ when $n \leq 1$임을 본 적이 있다. 언제 이런 경우가 발생하는가를 생각해보면 계속해서 `merge_sort`를 호출해서 $2^{i} = n$이 같아지는 경우가 될 것이다. 그러면 이 때 $i = \log n$이고, 이를 적용하면

$t(n) \hspace{0.5cm} =2^{\log n}t(n/2^{\log n}) + (\log n)cn \\
 \hspace{1.1cm} = nt(1) + cn\log n \\
 \hspace{1.1cm} = nb + cn\log n$
 
 따라서 $t(n)$은 $O(n\log n)$이 된다.
 
### 12.2.5 Alternative Implementations of Merge-Sort
### Sorting Linked Lists
merge-sort 알고리즘은 `list`를 이용하지 않고 기초적인 큐를 이용해서도 구현할 수 있다. `LinkedQueue`를 이용해서 $O(n\log n)$ bound를 갖게끔 구현해보자.


In [2]:
def merge(S1, S2, S):
    """Merge two sorted queue instances S1 and S2 into empty queue S."""
    while not S1.is_empty() and not S2.is_empty():
        if S1.first() < S2.first():
            S.enqueue(S1.dequeue())
        else:
            S.enqueue(S2.dequeue())
def merge_sort(S):
    """Sort the elements of queue S using the merge-sort algorithm."""
    n = len(S)
    if n < 2:
        return                              # list is already sorted
    # divide
    S1 = LinkedQueue()                      # or any other queue implementation
    S2 = LinkedQueue()
    while len(S1) < n // 2:                 # move the first n // 2 elements to S1
        S1.enqueue(S.dequeue())
    while not S.is_empty():                 # move the rest to S2
        S2.enqueue(S.dequeue())
    # conquer (with recursion)
    merge_sort(S1)                          # sort first half
    merge_sort(S2)                          # sort second half
    # merge results
    merge(S1, S2, S)                        # merge sorted halves back into S

<img width="800" alt="figure-12.7a" src="https://user-images.githubusercontent.com/20944657/37190148-9b27e694-239b-11e8-9033-fdca73ddbfea.png">

<img width="800" alt="figure-12.7b" src="https://user-images.githubusercontent.com/20944657/37190149-9b5c378c-239b-11e8-8930-c40a7d9edf34.png">

<img width="800" alt="figure-12.7c" src="https://user-images.githubusercontent.com/20944657/37190200-f2a8de00-239b-11e8-8429-5d92d69f7a5f.png">

### A Bottom-Up (Nonrecursive) Merge-Sort
재귀를 이용하지 않으면서도 $O(n \log n)$의 시간 복잡도를 갖는 배열 기반 merge-sort를 구현해보자. 이 경우 각각의 레벨에서 재귀 호출을 위한 오버헤드가 발생하지 않아서 재귀를 이용하는 merge-sort보다 조금 더 빠르다. 이 때 핵심적인 아이디어는 bottom-up으로, 트리의 아래부터 올라가면서 merge-sort를 실행하는 것이다. 위에서 다뤘던 linked list의 경우에도 비슷한 bottom-up 접근 방식을 취할 수 있다.

In [8]:
def merge(src, result, start, inc):
    """Merge src[start:start+inc] and src[start+inc:start+2*inc] into result."""
    end1 = start+inc                         # boundary for run 1
    end2 = min(start+2*inc, len(src))        # boundary for run 2
    x,y,z = start, start+inc, start          # index into run 1, run 2, result
    while x < end1 and y < end2:
        if src[x] < src[y]:
            result[z] = src[x]; x+= 1        # copy from run 1 and increment x
        else:
            result[z] = src[y]; y+= 1        # copy from run 2 and increment y
        z += 1                               # increment z to reflect new result
    if x < end1:
        result[z:end2] = src[x:end1]         # copy remainder of run 1 to output
    elif y < end2:
        result[z:end2] = src[y:end2]         # copy remainder of run 2 to output
        
def merge_sort(S):
    """Sort the elements of Python list S using the merge-sort algorithm."""
    n = len(S)
    logn = math.ceil(math.log(n,2))
    src, dest = S, [None] * n                # make temporary storage for dest
    for i in (2**k for k in range(logn)):    # pass i creates all runs of length 2i
        for j in range(0, n, 2*i):           # each pass merges two length i runs
            merge(src, dest, j, i)
        src, dest = dest, src                # reverse roles of lists
    if S is not src:
        S[0:n] = src[0:n]                    # additional copy to get results to S

## 12.3 Quick-Sort
다음으로 살펴볼 정렬 알고리즘은 **quick-sort**이다. merge-sort와 비슷하게 이 알고리즘도 **divide-and-conquer** 패러다임을 이용하지만 merge-sort와는 꽤나 다른 방식으로 이 테크닉을 사용한다. merge-sort에서는 재귀 호출 안에서 어려운 작업들이 일어나지만, quick-sort에서는 재귀 호출이 이루어지기 전에 어려운 작업들을 한다.

### High-Level Description of Quick-Sort
퀵 소트 알고리즘은 간단한 재귀 알고리즘을 이용해서 시퀀스 S를 정렬한다. 핵심적인 아이디어는 분할 정복 알고리즘을 이용해서 S를 부분 시퀀스로 분할하고, 각각의 subsequence를 정렬하게끔 재귀 호출을 실시한다. 그러고 나서 그 정렬된 subsequence를 병합하여 정렬을 마친다. quick-sort 알고리즘은 다음의 세 스텝으로 구성된다:
1. **Divide:** $S$가 최소 두 개의 원소를 갖는다면($S$가 비어있거나 하나의 원소만 가질 경우 아무 작업도 할 필요가 없다), $S$에서 **피벗(pivot)**이라 부르는 특정 원소 $x$를 고른다. 보통은 $S$의 마지막 원소 $x$를 피벗으로 정한다. 이제 $S$의 모든 원소를 다음의 세 시퀀스로 나눠준다:
    - $x$보다 작은 원소를 저장하는 $L$
    - $x$와 같은 원소를 저장하는 $E$
    - $x$보다 큰 원소를 저장하는 $G$

2. **Conquer:** $L$과 $G$ 시퀀스를 재귀적으로 정렬한다.
3. **Combine:** $L$의 원소를 $S$에 추가하고, $E$를 추가하고, $G$를 추가한다.

<img width="300" alt="figure-12.8" src="https://user-images.githubusercontent.com/20944657/37192704-47df8dde-23aa-11e8-8245-53036c004b4f.png">

merge-sort와 마찬가지로 quick-sort의 실행 또한 이진 트리를 이용해서 **quick-sort tree**로 나타낼 수 있다. 그러나 불행한 점이 있다면 quick-sort 트리의 높이는 최악의 경우 linear 해질 수도 있다. 예를 들어 이미 시퀀스가 이미 정렬이 되어있어서 마지막 원소를 피벗으로 정한 경우 높이가 $n-1$이 될 것이다. quick-sort의 실행 과정은 아래의 그림을 참고하자.
<img width="400" alt="figure-12.9a" src="https://user-images.githubusercontent.com/20944657/37192979-70edd9a0-23ab-11e8-8267-c30973a8eba6.png">
<img width="400" alt="figure-12.9b" src="https://user-images.githubusercontent.com/20944657/37192980-711812c4-23ab-11e8-86ea-63aa6d03caa2.png">

<img width="700" alt="figure-12.10ab" src="https://user-images.githubusercontent.com/20944657/37193308-99341d7e-23ac-11e8-9ae6-9a4e28ad88d6.png">
<img width="700" alt="figure-12.10cd" src="https://user-images.githubusercontent.com/20944657/37193309-9962d862-23ac-11e8-8eb3-d888b770c97c.png">
<img width="700" alt="figure-12.10ef" src="https://user-images.githubusercontent.com/20944657/37193310-99939ff6-23ac-11e8-8110-d2a8e133fe63.png">
<img width="700" alt="figure-12.11gh" src="https://user-images.githubusercontent.com/20944657/37193311-99c00460-23ac-11e8-96fb-20dda8ef9263.png">
<img width="700" alt="figure-12.11ij" src="https://user-images.githubusercontent.com/20944657/37193312-99f86b98-23ac-11e8-888e-82bfe57a2991.png">
<img width="700" alt="figure-12.11kl" src="https://user-images.githubusercontent.com/20944657/37193313-9a282b94-23ac-11e8-8cc1-ae8d8bdd41e5.png">
<img width="700" alt="figure-12.12mn" src="https://user-images.githubusercontent.com/20944657/37193314-9a599e9a-23ac-11e8-8c27-4fa19fe1775b.png">
<img width="700" alt="figure-12.12op" src="https://user-images.githubusercontent.com/20944657/37193315-9a89f126-23ac-11e8-832e-02357823cb2b.png">
<img width="700" alt="figure-12.12qr" src="https://user-images.githubusercontent.com/20944657/37193316-9ab653c4-23ac-11e8-8dbd-e0e95a4046ff.png">

### Performing Quick-Sort on General Sequences
이제 큐로 구현된 어떠한 시퀀스 타입에 대해 작동하는 quick-sort 알고리즘을 구현해보자. 아래의 코드에서 큐가 링크드 리스트로 구현되었다면 모든 큐 연산은 $O(1)$ worst-case time을 가질 것이다.

In [9]:
def quick_sort(S):
    """Sort the elements of queue S using the quick-sort algorithm."""
    n = len(S)
    if n < 2:
        return                          # list is already sorted
    # divide
    p = S.first()                       # using first as arbitrary pivot
    L = LinkedQueue()
    E = LinkedQueue()
    G = LinkedQueue()
    while not S.is_empty():             # divide S into L, E, and G
        if S.first() < p:
            L.enqueue(S.dequeue())
        elif p < S.first():
            G.enqueue(S.dequeue())
        else:                           # S.first() must equl pivot
            E.enqueue(S.dequeue())
            
    # conquer
    quick_sort(L)                       # sort elements less than p
    quick_sort(G)                       # sort elements greater than p
    # concatenate results
    while not L.is_empty():
        S.enqueue(L.dequeue())
    while not E.is_empty():
        S.enqueue(E.dequeue())
    while not G.is_empty():
        S.enqueue(G.dequeue())

### Running Time of Quick-Sort
merge-sort와 마찬가지 방법으로 시간 복잡도를 계산할 수 있다. 각각의 노드에 걸리는 시간을 구한 후 전체 노드에 대해 더하면 된다. 위의 코드를 잘 보면 분할과 병합 과정에서는 그 시간이 노드 $v$의 입력 사이즈 $s(v)$에 선형 관계를 가질 것임을 알 수 있다. 그런데 $E$가 항상 최소 하나의 원소(피벗)를 가지므로 $v$의 자식들의 입력 크기의 합은 최대 $s(v) - 1$이다.

$s_{i}$를 깊이 $i$에 있는 모든 노드들의 입력 크기의 합이라 하자. 그러면 $s_{0} = n$이고, $s_{1} \leq n-1$이다. 더 일반적으로 보면 $s_{i} \lt s_{i-1}$인데, 깊이가 $i-1$인 노드들의 피벗들은 $i$로 내려오지 않기 때문이다. 따라서 이를 정리하면 quick-sort 실행의 전체적인 작동 시간은 $O(n\cdot h)$임을 알 수 있다. 그런데 불행히도 최악의 경우(이미 시퀀스가 정렬되어있는 경우) quick-sort 트리의 높이는 $\Theta(n)$이 되어 quick-sort가 $O(n^{2})$ worst-case time을 갖는다. 정렬이 제일 쉬워야 할 경우가 최악의 경우가 되니 참 역설적이라 할 수 있겠다.

이름으로만 보면 quick-sort는 굉장히 빠를 것처럼 보인다. 그리고 실제로도 그렇다. quick-sort가 가장 빠를 때는 $L$과 $G$가 비슷한 크기를 갖게끔 나눠지는 경우이다. 이렇게 되면 트리의 높이는 $O(\log n)$이 되고 시간 복잡도가 $O(\log n)$이 될 것이다. 만약 $L$과 $G$의 분할이 완벽하지 않더라도 시간 복잡도는 $O(\log n)$이 될 수 있다. 예를 들어 매 스텝마다 하나의 subsequence가 $\frac{1}{4}$ 원소를 갖고 다른 subsequence가 $\frac{3}{4}$ 원소를 갖는다면 평균적으로 트리의 높이가 $O(\log n)$이 될 것이고, 전체 성능은 $O(n \log n)$이 될 것이다.

### 12.3.1 Randomized Quick-Sort
quick-sort에서는 시퀀스를 얼추 비슷한 크기를 가진 두 시퀀스로 분리하는 것이 중요하다. 이를 위해서 피벗을 가장 오른쪽의 원소로 정하는 것이 아니라 **랜덤**하게 뽑는 방법을 도입해보자. 이렇게 피벗을 랜덤으로 정할 경우 **randomized quick-sort**를 한다고 말한다. 다음의 명제는 randomized quick-sort의 예상 작동 시간이 $O(\log n)$임을 보여준다.

**Proposition 12.3:** 크기가 $n$인 시퀀스에 randomized quck-sort을 실행할 때의 예상 작동 시간은 $O(n\log n)$이다.

**Justification:** $S$의 두 원소들이 $O(1)$ 시간 안에 비교될 수 있다고 가정하자. 우리는 서브시퀀스 $L$과 $G$의 크기가 각각 최소 $n/4$, $3n/4$일 때 그 재귀 호출을 "좋은(good)" 호출이라 하고, 그렇지 않은 경우의 호출을 "나쁜(bad)" 호출이라 한다. 피벗을 랜덤하게 뽑을 경우 1/2의 확률로 좋은 호출이 일어나고 1/2의 확률로 나쁜 호출이 일어난다.  좋은 호출의 경우 최소한 시퀀스가 $n/4$, $3n/4$로 나눠지고, 나쁜 호출은 최악의 경우 시퀀스를 $0$, $n-1$로 나누게 된다. 이제 randomized quick-sort tree $T$의 recursion trace를 보자.

우리는 $T$의 노드 $v$의 subproblem이 $(3/4)^{i+1}$보다 크면서 $(3/4)^{i}n$보다 작을 때 $v$가 **size group** $i$에 속한다고 말한다. 이제 size group $i$에 속하는 모든 노드들에 대한 평균 시간을 구해보자. size group $i$에 속하는 노드 중 일부에 대해서는 good call이 이루어질 것이고 일부에 대해서는 bad call이 이루어질 것이다. good call이 일어날 경우 그 자식 노드들은 $i$보다 큰 size group에 속하게 된다. 그런데 bad call의 확률이 $1/2$이므로 good call이 이루어질때까지 필요한 호출의 평균 횟수는 2이다.

size group $i$에 있는 노드들은 각각의 호출(call)을 의미한다. 처음 입력받은 시퀀스 중 임의의 원소 $x$를 잡자. 그러면 size group $i$에 $x$를 포함하는 호출의 평균 개수는 2개이다. bad call이 일어날 경우 다음 size group으로 넘어가는 것이 아니라 그 size group에 여전히 있을 것을 생각하면 평균적으로 2번의 호출에 의해서 다음 size group으로 넘어가게 될 것이기 때문이다. 이러한 점을 고려한다면 size group $i$의 subproblem의 평균 총 사이즈는 $2n$이 된다. 각각의 재귀 호출에서는 상수번의 연산을 할 뿐이므로 size group $i$에서의 평균 작동 시간은 $O(n)$이다.

그러면 이제 size group의 총 개수만 알면 되는데, 반복적으로 $3/4$를 곱하는 것은 $4/3$으로 나누는 것과 같으므로 $\log_{4/3} n$이 총 개수가 된다. 따라서 size group의 수는 $O(\log n)$이고, randomized quick-sort의 총 평균 예상 작동시간은 $O(n\log n)$이다.

<img width="640" alt="figure-12.13" src="https://user-images.githubusercontent.com/20944657/37216724-dcb709c4-23fe-11e8-8754-ac2aa822800f.png">

### 12.3.2 Additional Optimizations for Quick-Sort
입력된 데이터를 위해 필요한 메모리 외에 거의 메모리를 사용하지 않는 알고리즘을 **in-place**라고 부른다. Section 9.4.2에서 heap-sort를 설명할 때 in-place 정렬 알고리즘을 이용한 적이 있다. 위에서 구현한 quick-sort는 추가적인 컨테이너 $L$, $E$, $G$를 필요로 하므로 in-place라 할 수 없다. 이제 in-place quick-sort를 구현한 아래의 코드를 보자.

In [1]:
def inplace_quick_sort(S, a, b):
    """Sort the list from S[a] to S[b] inclusive using the quick-sort algorithm."""
    if a >= b: return
    pivot = S[b]
    left = a
    right = b-1
    while left <= right:
        # scan until reaching value equal or larger than pivot (or right marker)
        while left <= right and S[left] < pivot:
            left += 1
        # scan until reaching value equal or smaller than pivot (or left marker)
        while left <= right and pivot < S[right]:
            right -= 1
        if left <= right:
            S[left], S[right] = S[right], S[left]
            left, right = left + 1, right - 1
    
    # put pivot into its final place (currently makred by left index)
    S[left], S[b] = S[b], S[left]
    # make recursive calls
    inplace_quick_sort(S, a, left-1)
    inplace_quick_sort(S, left+1, b)

<img width="500" alt="figure-12.14" src="https://user-images.githubusercontent.com/20944657/37217652-7309a718-2401-11e8-8ea8-1823ab68dd51.png">

### Pivot Selection
위의 구현에서는 피벗을 각각의 quick-sort recursion에서 가장 마지막 원소로 선택했지만, 이는 역시나 $\Theta(n^{2})$ worst-case time을 갖게 될 위험이 있다. Section 12.3.1에서 설명했듯이 피벗을 랜덤하게 택하면 이러한 문제를 피할 수 있지만, 현실에서 많이 사용하는 방법은 **median-of-three**라는 방법인데, 이 방법에서는 배열의 처음, 중간, 마지막에서 세 원소를 골라 그 중위값(median)을 피벗으로 이용한다. 이 방법을 이용하면 실제로도 더 좋은 결과가 나올 뿐 아니라 랜덤 넘버를 생성하기 위한 오버헤드가 필요하지 않다는 장점이 있다.

### Hybrid Approaches
quick-sort가 큰 데이터 셋에서 성능이 좋긴 하지만 상대적으로 크기가 작은 데이터 셋에서는 성능이 좋지 않다. 이는 위에서 겨우 8개의 원소를 저장하는 시퀀스를 정렬하기 위해 꽤나 긴 과정이 필요했던 것을 봐도 알 수 있다. 현실에서는 크기가 작은 시퀀스에 대해서는 insertion-sort와 같은 다른 정렬 알고리즘이 속도가 더 빠른 경우가 많다. 따라서 실제로 정렬을 구현할때는 size의 threshold를 정해서 그 값보다 크기가 클 때는 quick-sort를, 크기가 작을 때는 insertion-sort를 이용하는 경우가 많다. 자세한 것은 정렬 알고리즘의 비교와 현실적인 고려사항 등을 다루는 Section 12.5를 참고하자.

## 12.4 Studying Sorting through an Algorithmic Lens
### 12.4.1 Lower Bound for Sorting
정렬을 $O(n\log n)$보다 빠르게 하는 알고리즘이 있을까? 만약 정렬 알고리즘이 두 원소의 비교에 기반하여 이루어진다면 그 정렬 알고리즘은 $\Omega (n\log n)$ worst-case lower bound를 갖는다. 왜 그런지 알아보자. $S = (x_{0}, x_{1}, ..., x_{n-1})$을 정렬하려는 경우를 생각해보자. 정렬 알고리즘이 $x_{i}$와 $x_{j}$를 비교하는 것은 $x_{i} \lt x_{j}?$ 의 질문을 하는 것과 같고, 그 결과는 "yes"나 "no"일 수 밖에 없다. 이를 이용하면 우리는 원소의 비교를 이용하는 정렬 알고리즘을 decision tree $T$로 나타낼 수 있다. 이 $T$의 내부 노드 $v$는 비교(comparison)를 의미하고, 엣지들은 그 질문에 대한 답인 "yes"와 "no"를 의미한다. 이 때 중요한 것은 정렬 알고리즘이 $T$에 대한 명시적 지식을 갖지 않는다는 것이다. 즉, 트리는 정렬 알고리즘이 실시할 수 있는 모든 경우의 비교에 대한 정보를 담고 있다.

임의의 정렬 알고리즘은 $S$의 가능한 모든 초기 순서(initial order), 혹은 **순열(permutation)**에 대해 비교 연산을 실시하며 $T$의 루트부터 외부 노드까지의 경로를 따라 이동하게 된다. 이제 외부 노드 $v$와 정렬 알고리즘이 $v$에서 끝나게끔 하는 $S$의 순열의 집합을 연결지어보자. 그러면 중요한 발견을 하나 할 수 있는데, 외부 노드 $v$는 $S$의 순열 중 하나와 일대일 매칭이 된다는 것이다. 만약 $S$의 순열 $P_{1}$, $P_{2}$가 같은 외부 노드와 연결되어있다고 하자. 그러면 $P_{1}$에서는 $x_{i} \lt x_{j}$이지만 $P_{2}$에서는 $x_{j} \lt x_{i}$인 $x_{i}, x{j}$가 적어도 하나는 존재할 것이다. 그런데 정렬 알고리즘이 이 두 경우에 대해 똑같은 $v$로 결과를 냈다는 것은 정렬 알고리즘이 제대로 정렬을 하지 못하고 있음을 의미한다. 따라서 $v$는 오직 하나의 순열과 연결될 수 있다. 그러면 다음의 명제를 보자.

**Proposition 12.4:** $n$개의 원소를 갖는 시퀀스를 정렬하기 위한 비교-기반 알고리즘의 작동 시간은 worst-case에 $\Omega(n\log n)$이다.

**Justification:** 비교-기반 정렬 알고리즘의 작동 시간은 이 알고리즘을 나타내는 decision tree $T$의 높이보다 같거나 커야한다. 이는 $T$와 $v$의 의미를 생각하면 당연한데, $v$에 도달했을 때가 정렬이 끝난 경우이고, 정렬을 할때는 원소의 비교만 하는 것이 아니라 배열이나 링크드 리스트 내에서 원소를 이동하거나 하는 추가적인 작업도 이루어져야 하므로 필요한 시간이 $T$의 높이보다 크게 되는 것이다. 이제, 위에서 본 바와 같이 $T$의 외부 노드는 $S$의 순열 하나와 연결되어야 한다. 또, 각각의 순열은 반드시 $T$의 다른 외부 노드로 이어져야 한다. $n$개의 객체에 대한 순열의 개수는 $n!$이고, $T$는 적어도 $n!$개의 외부 노드를 갖는다. 그러면 명제 8.8에 의해 $T$의 높이는 적어도 $\log(n!)$이다. 

그런데, $n, n-1, ..., 2, 1$에는 $x \geq n/2$인 $x$가 적어도 $n/2$개가 있으므로 다음의 식이 성립하고, 증명이 끝난다.

$\log(n!) \geq \log((\dfrac{n}{2})^{\frac{n}{2}}) = \dfrac{n}{2}\log\dfrac{n}{2}$, which is $\Omega(n\log n)$

<img width="640" alt="figure-12.15" src="https://user-images.githubusercontent.com/20944657/37236081-a345b890-2447-11e8-8174-7125bf4b96f9.png">

### 12.4.2 Linear-Time Sorting: Bucket-Sort and Radix-Sort
앞에서 비교-기반 정렬 알고리즘의 경우 worst case에 $\Omega(n\log n)$의 시간이 필요함을 보였다. 이제 자연스럽게 나오는 의문은, 비교-기반이 아니라면 $O(n\log n)$ 보다 빠른 정렬 알고리즘을 설계하는 것이 가능한가에 대한 것이다. 흥미롭게도 그런 알고리즘이 존재하긴 하지만, 정렬을 실시할 시퀀스에 대한 가정이 필요하다. 그런데, 이미 알려진 범위의 정수를 정렬해야 한다던가, 문자열을 정렬한다던가 하는 그런 가정은 현실에서 자주 성립하므로 그런 정렬 알고리즘들을 살펴볼만한 가치가 있다. 이번 섹션에서는 key가 제한된 타입일 때 시퀀스를 정렬하는 문제를 살펴볼 것이다.

### Bucket Sort
비교에 기반하지 않은 알고리즘인 **bucket-sort** 알고리즘을 살펴보자. 이 알고리즘은 key가 정수 $N\geq2$일 때 $[0,N-1]$ 안에 있는 경우 사용할 수 있다. 그러면 $S$의 정렬이 $O(n+N)$ 시간 안에 이루어지고, $N$이 $O(n)$이면 전체 정렬 시간이 $O(n)$이 되어 굉장히 빠르게 이루어진다.

<img width="640" alt="bucket-sort" src="https://user-images.githubusercontent.com/20944657/37236209-c84f42d6-2448-11e8-9314-94a7324de47a.png">

### Stable Sorting
key-value 페어를 정렬할 때 중요한 것은 같은 key를 다루는 방법이다. $S=((k_{0},v_{0}),...,(k_{n-1},v_{n-1}))$을 정렬하는 경우를 생각해보자. 만약 정렬 전에 $k_{i} = k_{j}$이면서 $(k_{i},v_{i})$가 $(k_{j},v_{j})$ 앞에 있었는데 정렬 후에도 그 순서가 유지되는 경우 정렬 알고리즘이 **안정적(stable)**이라고 한다. 이러한 안정성은 실제 응용에서도 중요한데, 우리는 같은 key를 갖는 원소들간의 순서가 변화하는 것을 원하지 않는 경우가 많기 때문이다.

그렇다면 위의 bucket-sort는 stable할까? 만약 알고리즘의 구현에 쓰이는 시퀀스들이 모두 큐로 구현될 경우 stable하다고 할 수 있다. FIFO에 의해 먼저 들어간 (key,value) 페어가 먼저 나오게 되므로 순서가 유지될 것이다.

### Radix-Sort
stable sorting이 중요한 이유는 bucket-sort가 stable하면 정수 뿐 아니라 다른 일반적인 상황에 대해서도 적용될 수 있게 되기 때문이다.예를 들어 $(k,l)$ 페어를 정렬하고 싶은데 $k$와 $l$이 $[0,N-1]$의 범위에 속하는 정수인 경우를 생각해보자. 이럴 때는 이러한 key들을 **사전편찬식(lexicographic)**으로 정렬하는 것이 일반적이다. 

**radix-sort** 알고리즘은 이렇게 key가 페어로 이루어진 경우 stable bucket-sort를 여러번 적용해서 시퀀스 $S$를 정렬하는 알고리즘이다. 여기서 중요한 것은 bucket-sort를 어떤 key에 먼저 적용해야 하는 것의 문제인데, 이 문제에 답하기 위해 다음의 예제를 보자.

$S = ((3, 3), (1, 5), (2, 5), (1, 2), (2, 3), (1, 7), (3, 2), (2, 2))$

위의 예제의 페어는 (key1, key2)로 이루어진 것이고, value는 생략하였다. 먼저 key1에 bucket-sort를 적용하고 key2에 bucket-sort를 적용해보자.

$S_{1} = ((1, 5), (1, 2), (1, 7), (2, 5), (2, 3), (2, 2), (3, 3), (3, 2))\\
S_{1,2} = ((1, 2), (2, 2), (3, 2), (2, 3), (3, 3), (1, 5), (2, 5), (1, 7))$

안타깝게도 우리가 원하는 결과가 나오지 않았다. 그러면 이제 key2에 먼저 bucket-sort를 적용해보자.

$S_{2} = ((1, 2), (3, 2), (2, 2), (3, 3), (2, 3), (1, 5), (2, 5), (1, 7))\\
S_{2,1} = ((1, 2), (1, 5), (1, 7), (2, 2), (2, 3), (2, 5), (3, 2), (3, 3))$

우리가 원했던 것처럼 lexicographic하게 결과가 잘 나온 것을 확인할 수 있다. 간단한 예제이지만, 일반적으로도 두번째 component에 stable sorting을 적용하고 나서 첫번째 component에 stable sorting을 적용하면 lexicographic하게 정렬할 수 있음을 보일 수 있다. 이제 다음의 명제를 보자.

**Proposition 12.6:** $S$를 $n$개의 key-value 페어를 가진 시퀀스라 하자. 각각의 페어는 정수 [0,N-1] 범위 안의 key $(k_{1}, k_{2}, ..., k_{d})$를 갖는다. 그러면 우리는 radix-sort를 이용해서 $O(d(n+N))$ 시간 안에 $S$를 lexicographic하게 정렬할 수 있다.

radix-sort는 그 길이가 제한된 문자열에도 적용할 수 있는데, 각각의 문자를 정수로 표현할 수 있기 때문이다(서로 다른 길이의 문자열을 다루기 위해 위의 예제보다는 조금 더 신경을 써야 할 필요가 있다).

## 12.5 Comparing Sorting Algorithms
### Considering Running Time and Other Factors
우리는 지금까지 $O(n^{2})$ average, worst-case time을 갖는 insertion-sort, selection-sort를 살펴봤고, $O(n\log n)$ time을 갖는 heap-sort, merge-sort, quick-sort를 알아봤다. 또, 일부 한정된 타입의 key에 대해 linear time을 갖는 bucket-sort와 radix-sort도 알아보았다. 이 중에서 selection-sort 알고리즘은 best case에도 $O(n^{2})$의 시간이 걸리므로 어느 경우에도 사용하지 않을 것이 분명하다. 그러나 나머지 알고리즘 중에서는 어떤 알고리즘이 제일 좋은 알고리즘일까?

여러 알고리즘 중에 "제일 좋은" 알고리즘이란 건 존재하지 않는다. 효율성, 메모리 이용, 안정성의 측면에서 여러 trade-off가 존재한다. 어떤 상황에 어떤 정렬 알고리즘을 써야 할 지는 모두 다르기 때문에 각각의 알고리즘의 장단점에 대해 알아보도록 하자.

### Insertion-Sort
적절히 구현하기만 하면 **insertion-sort**의 작동 시간은 $O(n+m)$이 되는데, $m$은 **inversion**(순서가 맞지 않는 원소 쌍의 개수)의 수를 의미한다. 따라서 insertion-sort는 원소가 50개 이하인 크기가 작은 시퀀스를 정렬하기에 훌륭한 알고리즘이다. 또, insertion-sort는 "거의" 정렬된 시퀀스에서 매우 효율적이다. "거의" 정렬되었다는 것은 inversion의 수가 적음을 의미한다. 그러나 insertion-sort의 $O(n^{2})$ 성능으로 인해 사실상 insertion-sort를 특정한 경우 외에 사용하기는 힘들다.

### Heap-Sort
heap-sort는 worst case에도 $O(n\log n)$ 안에 실행된다. heap-sort는 in-place로 작동되게끔 구현하기가 쉽고, 메모리 안에 다 저장될 수 있는 적당한 사이즈의 시퀀스를 정렬하기에 최적의 정렬 알고리즘이다. 그러나 큰 규모의 시퀀스에서는 quick-sort와 merge-sort가 heap-sort보다 성능이 좋다. 표준적인 heap-sort는 원소의 스왑으로 인해 stable하지 않다.

### Quick-Sort
worst-case의 성능이 $O(n^{2})$이기 때문에 정렬 알고리즘의 성능에 대한 확실한 보장이 필요한 경우에는 quick-sort를 사용할 수 없다. 그러나 quick-sort의 평균 성능이 $O(n\log n)$이고, 실험 연구에 따르면 quick-sort가 많은 경우 heap-sort나 merge-sort보다 빠르다고 알려져있기 때문에 quick-sort가 자주 이용된다. quick-sort는 시퀀스를 분할하는 과정에서 원소를 스왑하기 때문에 stable하지 않다.

지난 수십년간 일반적인 목적의 in-memory 정렬 알고리즘으로는 거의 대부분 quick-sort를 이용해왔다. quick-sort는 `C` 언어 라이브러리에서 제공되는 `qsort` 정렬 유틸리티로 포함되어있으며, 유닉스 운영체제의 정렬에서도 이용되어왔다. 또, Java6까지 quick-sort는 배열 정렬의 표준적인 알고리즘으로 이용되기도 하였다.

### Merge-Sort
**merge-sort**는 worst-case에도 $O(n\log n)$ 시간이 걸린다. merge-sort를 배열에서 in-place로 작동하게끔 구현하는 것은 꽤나 어렵다. 추가적인 배열을 할당하여 배열 간의 복사를 해야하는 merge-sort는 메인 메모리의 용량에 딱 맞게끔 in-place로 정렬을 해주는 heap-sort와 quick-sort에 비교하면 매력적이지 않다. 그럼에도 불구하고 merge-sort는 입력이 컴퓨터의 메모리 구조 내부의 다양한 계층 내부(캐쉬, 메인 메모리, 외부 메모리)에 존재하는 경우에는 훌륭한 알고리즘이 된다. 이러한 경우 merge-sort는 각각의 메모리 계층 안의 데이터를 긴 merge stream으로 관리하여 메모리 계층 사이의 transfer를 줄일 수 있다.

### Bucket-Sort and Radix-Sort
작은 정수, 문자열, 특정 범위 내부의 key로 이루어진 d-tuple 을 정렬해야 하는 경우에 **bucket-sort** **radix-sort**는 $O(d(n+N))$ 시간 안에 정렬을 끝낼 수 있다. $d(n+N)$은 $n\log n$에 비해 매우 작은 수치이고, quick-sort, heap-sort, merge-sort에 비해 매우 빠르게 작동한다.

## 12.6 Python's Built-In Sorting Functions
파이썬은 데이터를 정렬하기 위한 두가지 방법을 제공한다. 첫번째는 `list` 클래스의 `sort` 메소드이다. 예를 들어 아래와 같은 리스트가 있다고 하자.

`colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow']`

`sort` 메소드는 원소들이 갖는 `<` 연산자의 자연스러운 의미에 따라 원소들을 정렬한다. 위의 예에서는 원소들이 문자열이고, 문자열의 순서는 알파벳의 순서에 의해 정의된다. 따라서 `colors.sort()`를 호출하면 리스트는 다음과 같이 될 것이다.

`['blue', 'cyan', 'green', 'magenta', 'red', 'yellow']`

파이썬은 또한 built-in 함수인 `sorted`를 지원하는데, iterable 컨테이너의 원소들을 정렬한 새로운 리스트를 반환해준다. 예를 들어 위의 `colors` 리스트에 `sorted(colors)`를 호출하면 `colors` 리스트에는 변화가 없이 새로 정렬된 리스트가 반환될 것이다. 실제로는 `sorted`를 더 많이 사용하는데, 모든 iterable 객체를 파라미터로 받기 때문이다. 예를 들어 `sorted(green`)은 `['e','e','g','n','r']`을 반환한다.

### 12.6.1 Sorting According to a Key Function.
가끔은 원소의 `<` 연산자가 아닌 다른 기준으로 정렬을 해야 할 때가 있다. 파이썬의 built-in `sort` 함수는 매개변수로 key 함수를 받아 호출자로 하여금 정렬할 순서를 정의할 수 있게 한다. 이 때 key 함수는 원소를 인자로 받아 key를 반환하는 함수여야 한다. 예를 들어 `colors.sort(key=len)`나 `sorted(colors, key=len)`을 실행하면 알파벳 순서가 아니라 각 문자열의 길이에 의해 정렬이 되고, 다음과 같이 정렬될 것이다.

`['red', 'blue', 'cyan', 'green', 'yellow', 'magenta']`

정렬 함수는 또한 `reverse`라는 파라미터를 받는데, `True`일 경우 거꾸로 정렬이 이루어진다.

### Decorate-Sort-Undecorate Design Pattern
파이썬에서 key 함수를 이용해서 정렬을 할 수 있게끔 한 것은 **decorate-sort-undecorate design pattern**을 이용한 것이다. 이 디자인 패턴은 다음의 3 스텝으로 이루어진다:
1. key 함수를 원소에 적용한 결과를 포함하게끔 리스트의 원소를 "장식된(decorated)" 버전으로 교체한다.
2. key의 자연스러운 순서에 따라 리스트를 정렬한다
3. decorate된 원소를 다시 원래의 원소로 교체한다.

아래는 문자열의 길이를 decoration으로 이용해서 원소를 decorate하는 예제이다.

<img width="500" alt="figure-12.16" src="https://user-images.githubusercontent.com/20944657/37237331-230d1520-2455-11e8-801e-ab0cb95180d0.png">

이미 파이썬에 구현이 되어있긴 하지만 이를 직접 구현한다면 우선순위 큐에서 key-value 페어를 이용한 것과 같은 컴포지트 패턴을 이용해야 할 것이다.

In [1]:
def decorated_merge_sort(data, key=None):
    """Demonstration of the decorate-sort-undecorate pattern."""
    if key is not None:
        for j in range(len(data)):
            data[j] = _Item(key(data[j]), data[j])  # decorate each element
    merge_sort(data)                                # sort with existing algorithm
    if key is not None:
        for j in range(len(data)):
            data[j] = data[j]._value                # undecorate each element

## 12.7 Selection
정렬을 하는 것도 중요하긴 하지만, 전체 집합 내에서 그 순서에 따라 원소 하나를 찾는 것도 실제 응용 사례에서는 중요한 문제이다. 이러한 문제의 사례로는 최소값, 최대값을 찾는 것이나, 중위값(median)을 찾는 것 등이 있다. 일반적으로 주어진 순서에 해당하는 원소를 요청하는 쿼리를 **순서 통계량(order statistics)**이라고 한다.

### Defining the Selection Problem
이번 섹션에서는 정렬되지 않은 $n$개의 비교 가능한 항목들로 이루어진 집합에서 $k^{th}$로 작은 원소를 찾는 일반적인 순서통계량 문제를 다룰 것이다. 이는 **선택(selection)** 문제로 알려져 있다. 물론 정렬을 한 후에 해당 인덱스의 원소를 고르는 쉬운 방법이 있긴 하지만 이 방법은 $O(n\log n)$의 시간 복잡도를 갖는다.그런데  $k=1$이거나 $k=n, k=2, k=3, k=n-1, k=n-5$와 같은 경우에는 $O(n)$ 시간 안에 원하는 값을 찾을 수 있는데도 굳이 정렬을 하는 것은 overkill이 된다. 이제 우리가 알고 싶은 것은 $k = \lfloor n/2 \rfloor$와 같은 중위값 문제 등 모든 $k$ 값에 대해 $O(n)$ 시간 복잡도를 갖게끔 할 수 있는가의 문제이다.

### 12.7.1 Prune-and-Search
놀랍게도 우리는 모든 $k$값에 대해 선택 문제를 $O(n)$ 시간 안에 푸는 것이 가능하다. 이를 위해서는 **prune-and-search** 혹은 **decrease-and-conquer**라고 알려진 흥미로운 알고리즘 디자인 패턴을 이용해야 한다. 이 디자인 패턴은 $n$개의 객체들의 집합에서 일부를 잘라내고 재귀적으로 더 작은 문제를 푸는데, 계속 크기를 줄여가다가 상수 개수의 집합을 만나게 되는 경우 brute-force로 문제를 해결한다. 일부 경우에는 재귀를 이용하지 않고 brute-force를 이용할 수 있을 때까지 prune-and-search reduction step을 iterate하는 것만으로도 문제를 해결할 수 있다. 이러한 prune-and-search 디자인 패턴의 예제로는 우리가 이미 많이 살펴봤던 이진 탐색 메소드를 들 수 있다.

### 12.7.2 Randomized Quick-Select
prune-and-search 패턴을 적용한 간단하고 실용적인 알고리즘인 **randomized quick-select**를 살펴보자. 이 알고리즘은 이 알고리즘이 이용하는 모든 random choice에 대해 $O(n)$ **expected** time을 갖는다. 이 expectation은 입력 데이터의 분포와는 전혀 관련이 없다. 이 randomized-quick-search는 **worst-case**에 $O(n^{2})$ 시간 복잡도를 갖는다. 연습문제 C-12.55에서는 이 randomized qucik-select를 수정해서 $O(n)$ **worst-case** 시간이 걸리는 **deterministic** 선택 알고리즘을 정의하는 법을 알아볼 수 있을 것이다.

이제 이 알고리즘을 어떻게 파이썬으로 구현하는 지 알아보자. 전체적인 방법은 Section 12.3.1에서 봤던 randomized qucik-sort 알고리즘과 유사하다.

In [2]:
def quick_select(S, k):
    """Return the kth smallest element of list S, for k from 1 to len(S)."""
    if len(S) == 1:
        return S[0]
    pivot = random.choice(S)           # pick random pivot element from S
    L = [x for x in S if x < pivot]    # elements less than pivot
    E = [x for x in S if x == pivot]   # elements qual to pivot
    G = [x for x in S if pivot < x]    # elements greater than pivot
    if k <= len(L):
        return quick_select(L,k)       # kth smallest lies in L
    elif k <= len(L) + len(E):
        return pivot                   # kth smallest equal to pivot
    else:
        j = k - len(L) - len(E)        # new selection parameter
        return quick_select(G, j)      # kth smallest is jth in G

### 12.7.3 Analyzing Randomized Quick-Select
이제 randomized quick-select가 $O(n)$의 시간 복잡도를 갖는다는 것을 보일 것이다. $t(n)$을 크기가 $n$인 시퀀스에 randomized quick-select를 실시할 때 걸리는 시간이라고 하자. 이 알고리즘은 확률 사건에 의존하므로 $t(n)$은 확률 변수가 된다. 우리는 $t(n)$의 평균인 $E(t(n))$을 bound하기를 원한다. 이제 우리 알고리즘의 재귀적 호출에서 $L$이나 $G$의 크기가 최대 $3n/4$이게끔 $S$의 분할이 이루어졌을때 이 호출을 "좋은(good)" 호출이라 하자. 그러면 재귀 호출은 최소 $1/2$의 확률로 good call일 것이다.

이제 $g(n)$을 good call을 실행하기 전 까지 우리가 만드는 연속적인 재귀 호출의 수라 하자. 그러면 우리는 $t(n)$을 다음과 같이 **점화식(recurrence equation)**을 이용해서 표현할 수 있다:

$t(n) \leq bn \cdot g(n) + t(3n/4)$, where $b \geq 1$ is a constant.

이제 $n>1$일 때 기대값을 취하면

$E(t(n)) \leq E(bn \cdot g(n) + t(3n/4)) = bn \cdot E(g(n)) + E(t(3n/4))$

재귀 호출 good일 확률은 최소 1/2이고, 각각의 호출이 good일 사건은 독립 사건이므로 $g(n)$의 기대값은 $E(g(n)) \leq 2$가 된다. 이제 편의를 위해 $T(n) = E(t(n))$이라 쓰면 우리는 위의 식을 아래와 같이 다시 쓸 수 있다.

$T(n) \leq T(3n/4) + 2bn$

이를 closed-form으로 바꾸기 위해 $n$이 크다고 가정하고 아래와 같이 점화식을 반복해서 적용해보자.

$T(n) \leq T((3/4)^{2}n) + 2b(3/4)n + 2bn$

그러면 다음의 일반항을 구할 수 있을 것이다.

$T(n) \leq 2bn \cdot \sum_{i=0}^{\lceil \log_{4/3}n \rceil} (3/4)^{i}$

그런데 오른쪽 항의 summation이 $r \lt 1$인 무한등비급수 $\times$ 2bn 이므로 $T(n)$이 $O(n)$라는 결론을 얻을 수 있다.

**Proposition 12.7:** 크기가 $n$인 시퀀스 $S$에 randomized quick-select를 실행할 때의 평균 작동 시간은 $O(n)$이다. 이 때 $S$의 두 원소의 비교가 $O(1)$ 시간에 이루어질 수 있다고 가정한다.